In [2]:
library(tictoc)
library(data.table)

# Inizio timer generale
tic("esercizio10")
atac_dt <- fread("atac_peaks.bed.csv")
gene_dt <- fread("gene_annotation.bed.csv")

#TASK1
tic("Task1")

#setnames(atac_dt, old = c("start", "end"), new = c("start", "end"))
setnames(gene_dt, old = c("start", "end"), new = c("i.start", "i.end"))

setkey(atac_dt, chr, start, end)
setkey(gene_dt, chr, i.start, i.end)

overlap_dt <- foverlaps(
    x = atac_dt, 
    y = gene_dt, 
    by.x = c("chr", "start", "end"), # Colonne ATAC
    by.y = c("chr", "i.start", "i.end"), # Colonne Gene
    type = "any", # Tipo di sovrapposizione
    nomatch = 0L # Rimuove le righe senza corrispondenza
)

setnames(overlap_dt, old = c("start", "end", "i.start", "i.end"), 
                     new = c("peak_start", "peak_end", "gene_start", "gene_end"))

toc()
cat("Risultato dell'intersezione:\n")
print(head(overlap_dt, 5))

#TASK2
tic("Task2")
peaks_per_gene_dt <- overlap_dt[, 
    .(num_peaks = uniqueN(peak_id)), # Conta i picchi ID unici
    by = gene
]
toc()
cat("\nConteggio dei picchi per gene \n")
print(head(peaks_per_gene_dt, 10))

#TASK3
tic("Task3")

overlap_dt[, overlap_start := pmax(peak_start, gene_start)]
overlap_dt[, overlap_end := pmin(peak_end, gene_end)]
overlap_dt[, overlap_bp := overlap_end - overlap_start]

total_overlap_bp_dt <- overlap_dt[, 
    .(total_overlap_bp = sum(overlap_bp)), 
    by = gene
]
toc()
cat("\nSomma totale di bp sovrapposti per gene \n")
print(head(total_overlap_bp_dt,5))

#TASK4
tic("Task4")

final_results_dt <- merge(total_overlap_bp_dt, peaks_per_gene_dt, by = "gene")

top_20_genes <- final_results_dt[order(-total_overlap_bp)][1:20]
#top_20_genes <- head(ordered_results_dt, 20)

toc()
cat("\nTOP 20 Geni per Lunghezza Totale di Sovrapposizione (bp):\n")
print(top_20_genes)

# Fine timer generale
toc()


Attaching package: ‘data.table’


The following object is masked from ‘package:tictoc’:

    shift




Task1: 0.031 sec elapsed
Risultato dell'intersezione:
Key: <chr, peak_start, peak_end>
      chr gene_start gene_end      gene peak_start peak_end    peak_id score
   <char>      <int>    <int>    <char>      <int>    <int>     <char> <int>
1:   chr1    1064323  1084034 GENE_0356    1067760  1068860 peak_02815   425
2:   chr1    1317537  1346385 GENE_0233    1325076  1325461 peak_04736   675
3:   chr1    1317537  1346385 GENE_0233    1337256  1338396 peak_00981   143
4:   chr1    1327359  1353543 GENE_0232    1337256  1338396 peak_00981   143
5:   chr1    1317537  1346385 GENE_0233    1341992  1343938 peak_04765   174
Task2: 0.019 sec elapsed

Conteggio dei picchi per gene 
         gene num_peaks
       <char>     <int>
 1: GENE_0356         1
 2: GENE_0233         4
 3: GENE_0232         4
 4: GENE_0049         1
 5: GENE_0245         2
 6: GENE_0130         1
 7: GENE_0477         2
 8: GENE_0054         4
 9: GENE_0427         1
10: GENE_0349         7
Task3: 0.013 sec elapsed

Som

In [3]:
library(tictoc)

# Inizio timer generale
tic("esercizio10_dataframe")

# Caricamento dati
atac_df <- read.csv("atac_peaks.bed.csv")
gene_df <- read.csv("gene_annotation.bed.csv")

# Rinominare le colonne per chiarezza
names(atac_df)[names(atac_df) == "X.chr"] <- "chr"
names(gene_df)[names(gene_df) == "X.chr"] <- "chr"

# TASK1: Intersezione degli intervalli (Sostituzione di foverlaps)
tic("Task1_dataframe")

# Prepara i data.frame per l'intersezione: devono avere le stesse colonne
# Si presume che le colonne start/end siano già presenti

# Implementazione manuale di foverlaps per data.frame
overlap_list <- list()
counter <- 1

# Esegue l'iterazione su ogni riga di atac_df
for (i in 1:nrow(atac_df)) {
  atac_row <- atac_df[i, ]
  
  # Filtra i geni sullo stesso cromosoma
  matching_genes <- gene_df[gene_df$chr == atac_row$chr, ]
  
  # Trova le sovrapposizioni (criterio 'any'):
  # start_atac <= end_gene E end_atac >= start_gene
  # Ricorda: gene_df ha colonne 'start' e 'end' originali.
  overlaps <- matching_genes[
    atac_row$start <= matching_genes$end & atac_row$end >= matching_genes$start, 
  ]
  
  # Se ci sono sovrapposizioni, crea le righe di risultato
  if (nrow(overlaps) > 0) {
    # Combina la riga ATAC con tutte le righe Gene sovrapposte
    for (j in 1:nrow(overlaps)) {
      gene_row <- overlaps[j, ]
      overlap_list[[counter]] <- data.frame(
        chr = atac_row$chr,
        peak_start = atac_row$start,
        peak_end = atac_row$end,
        peak_id = atac_row$peak_id,
        gene_start = gene_row$start,
        gene_end = gene_row$end,
        gene = gene_row$gene,
        stringsAsFactors = FALSE
      )
      counter <- counter + 1
    }
  }
}

# Combina tutti i risultati in un unico data.frame
overlap_df <- do.call(rbind, overlap_list)

toc()
cat("Risultato dell'intersezione (prime 5 righe):\n")
print(head(overlap_df, 5))

# ---

# TASK2: Conteggio dei picchi unici per gene
tic("Task2_dataframe")

# Utilizzo di aggregate per contare i peak_id unici per ogni gene
peaks_per_gene_df <- aggregate(
  overlap_df$peak_id, 
  by = list(gene = overlap_df$gene), 
  FUN = function(x) length(unique(x)) # Conta gli ID unici
)
names(peaks_per_gene_df)[names(peaks_per_gene_df) == "x"] <- "num_peaks"

toc()
cat("\nConteggio dei picchi per gene (prime 10 righe):\n")
print(head(peaks_per_gene_df, 10))

# ---

# TASK3: Calcolo della lunghezza totale di sovrapposizione
tic("Task3_dataframe")

# Calcola l'inizio e la fine della sovrapposizione
overlap_df$overlap_start <- pmax(overlap_df$peak_start, overlap_df$gene_start)
overlap_df$overlap_end <- pmin(overlap_df$peak_end, overlap_df$gene_end)

# Calcola la lunghezza della sovrapposizione in bp
overlap_df$overlap_bp <- overlap_df$overlap_end - overlap_df$overlap_start

# Utilizzo di aggregate per sommare overlap_bp per ogni gene
total_overlap_bp_df <- aggregate(
  overlap_df$overlap_bp, 
  by = list(gene = overlap_df$gene), 
  FUN = sum
)
names(total_overlap_bp_df)[names(total_overlap_bp_df) == "x"] <- "total_overlap_bp"

toc()
cat("\nSomma totale di bp sovrapposti per gene (prime 5 righe):\n")
print(head(total_overlap_bp_df, 5))

# ---

# TASK4: Unione dei risultati e TOP 20
tic("Task4_dataframe")

# Unisce i risultati dei task 2 e 3
final_results_df <- merge(total_overlap_bp_df, peaks_per_gene_df, by = "gene")

# Ordina e seleziona i TOP 20
final_results_df <- final_results_df[order(final_results_df$total_overlap_bp, decreasing = TRUE), ]
top_20_genes <- head(final_results_df, 20)

toc()
cat("\nTOP 20 Geni per Lunghezza Totale di Sovrapposizione (bp):\n")
print(top_20_genes)

# Fine timer generale
toc()

Task1_dataframe: 1.129 sec elapsed
Risultato dell'intersezione (prime 5 righe):
   chr peak_start peak_end    peak_id gene_start gene_end      gene
1 chr1    1067760  1068860 peak_02815    1064323  1084034 GENE_0356
2 chr1    1325076  1325461 peak_04736    1317537  1346385 GENE_0233
3 chr1    1337256  1338396 peak_00981    1317537  1346385 GENE_0233
4 chr1    1337256  1338396 peak_00981    1327359  1353543 GENE_0232
5 chr1    1341992  1343938 peak_04765    1317537  1346385 GENE_0233
Task2_dataframe: 0.008 sec elapsed

Conteggio dei picchi per gene (prime 10 righe):
        gene num_peaks
1  GENE_0000         2
2  GENE_0002         2
3  GENE_0003         1
4  GENE_0004         1
5  GENE_0005         3
6  GENE_0007         1
7  GENE_0008         1
8  GENE_0009         6
9  GENE_0010         5
10 GENE_0011         1
Task3_dataframe: 0.004 sec elapsed

Somma totale di bp sovrapposti per gene (prime 5 righe):
       gene total_overlap_bp
1 GENE_0000             1373
2 GENE_0002             

In [4]:
#tabella confronto tempi
T_DT1 <- 0.031
T_DT2 <- 0.019
T_DT3 <- 0.013
T_DT4 <- 0.007
T_DF1 <- 1.129
T_DF2 <- 0.008
T_DF3 <- 0.004
T_DF4 <- 0.004

# Creazione della tabella riassuntiva
risultati_performance <- data.frame(
  Task = c("Task 1",
           "Task 2",
           "Task 3",
          "Task 4"),
  
  Tempo_data.table_Sec = c(T_DT1, T_DT2, T_DT3, T_DT4),
  Tempo_data.frame_Sec = c(T_DF1, T_DF2, T_DF3, T_DF4)
)

# Aggiungiamo una colonna per il fattore di velocizzazione (Speedup)
risultati_performance$Speedup_DT_vs_DF <- 
  round(risultati_performance$Tempo_data.frame_Sec / risultati_performance$Tempo_data.table_Sec, 1)

# Stampiamo la tabella finale
print(risultati_performance)

    Task Tempo_data.table_Sec Tempo_data.frame_Sec Speedup_DT_vs_DF
1 Task 1                0.031                1.129             36.4
2 Task 2                0.019                0.008              0.4
3 Task 3                0.013                0.004              0.3
4 Task 4                0.007                0.004              0.6
